![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Trainining a Text Classification Model

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark


sparknlp.version : 3.1.0
sparknlp_jsl.version : 3.1.1
Out[3]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.1 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

## Load dataset

In [0]:
 !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/petfinder-mini.csv
 
 dbutils.fs.cp("file:/databricks/driver/petfinder-mini.csv", "dbfs:/") 

Out[5]: True

In [0]:
dataframe = pd.read_csv('petfinder-mini.csv')

In [0]:
# In the original dataset "4" indicates the pet was not adopted.

dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

In [0]:
dataframe = dataframe.drop(['AdoptionSpeed'], axis=1)

In [0]:
dataframe.head()

Out[9]:

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of ca...,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to p...,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresponsible owner at the roadside near some shops in S...,7,1
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience waiting for her good master, plz call or sms for m...",8,1
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption. He is the most playful pal we've seen in our pupp...,3,1


In [0]:
dataframe.columns

Out[10]: Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
 'PhotoAmt', 'target'],
 dtype='object')

In [0]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 Type 11537 non-null object
 1 Age 11537 non-null int64 
 2 Breed1 11537 non-null object
 3 Gender 11537 non-null object
 4 Color1 11537 non-null object
 5 Color2 11537 non-null object
 6 MaturitySize 11537 non-null object
 7 FurLength 11537 non-null object
 8 Vaccinated 11537 non-null object
 9 Sterilized 11537 non-null object
 10 Health 11537 non-null object
 11 Fee 11537 non-null int64 
 12 Description 11528 non-null object
 13 PhotoAmt 11537 non-null int64 
 14 target 11537 non-null int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB

In [0]:
dataframe.target.value_counts()

Out[12]: 1 8457
0 3080
Name: target, dtype: int64

In [0]:
dataframe.Description = dataframe.Description.fillna('- no description -')

## Featurize with Sklearn Column Transformer

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (OneHotEncoder(handle_unknown='ignore', categories='auto'), ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health']),
     (TfidfVectorizer(max_features=100,  norm='l2', ngram_range=(1, 3)), 'Description'),
     remainder=StandardScaler())

X = column_trans.fit_transform(dataframe.drop(['target'], axis=1))

y = dataframe.target

In [0]:
y.nunique()


Out[15]: 2

In [0]:
X.shape

Out[16]: (11537, 302)

In [0]:
input_dim = X.shape[1]

In [0]:
input_dim

Out[18]: 302

In [0]:
import scipy.sparse

df = pd.DataFrame.sparse.from_spmatrix(X)

df.columns = ['col_{}'.format(i) for i in range(input_dim)]

df = df.fillna(0)

df['target']= y

df.head()

Out[19]:

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_253,col_254,col_255,col_256,col_257,col_258,col_259,col_260,col_261,col_262,col_263,col_264,col_265,col_266,col_267,col_268,col_269,col_270,col_271,col_272,col_273,col_274,col_275,col_276,col_277,col_278,col_279,col_280,col_281,col_282,col_283,col_284,col_285,col_286,col_287,col_288,col_289,col_290,col_291,col_292,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.171825,0.0,0.137765,0.0,0.000000,0.283609,0.0,0.220992,0.151594,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.144907,0.000000,0.0,0.158869,0.0,0.0,0.000000,0.170202,0.000000,0.192465,0.190783,0.000000,0.132427,0.176639,0.151246,0.000000,0.0,0.000000,0.000000,0.257843,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.452479,0.950288,-0.829762,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.228484,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.256152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250843,0.000000,0.0,0.000000,0.000000,0.213817,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,-0.511871,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.141649,0.000000,0.0,0.000000,0.0,0.172449,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.307225,0.0,0.178193,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.210999,0.209155,0.209619,0.000000,0.000000,0.414527,0.179294,0.0,0.000000,0.000000,0.141336,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,1.077583,1
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.257886,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.198538,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.226480,0.0,-0.400729,1.575125,1.395473,1
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.126763,0.113263,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.081638,0.000000,0.105362,0.128186,0.0,0.000000,0.0,0.000000,0.107062,0.086649,0.0,0.000000,0.0,0.0,0.000000,0.125751,0.218471,0.142200,0.000000,0.000000,0.195683,0.000000,0.111746,0.120832,0.0,0.120993,0.000000,0.000000,0.223069,0.117846,0.0,0.198113,0.0,0.133816,0.181835,0.0,-0.555981,-0.299388,-0.193980,1


## Train with Spark NLP Generic Classifier

**Building a pipeline**

The FeaturesAssembler is used to collect features from different columns. It can collect features from single value columns (anything which can be cast to a float, if casts fails then the value is set to 0), array columns or SparkNLP annotations (if the annotation is an embedding, it takes the embedding, otherwise tries to cast the 'result' field). The output of the transformer is a FEATURE_VECTOR annotation (the numeric vector is in the 'embeddings' field).

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations. The operation of the classifier is controled by the following methods:

*setEpochsNumber(int)* - Determines how many epochs the model is trained.

*setBatchSize(int)* - Sets the batch size during training.

*setLearningRate(float)* - Sets the learning rate.

*setValidationSplit(float)* - Sets the proportion of examples in the training set used for validation.

*setModelFile(string)* - Loads a model from the specified location and uses it instead of the default model.

*setFixImbalance(boolean)* - If set to true, it tries to balance the training set by weighting the classes according to the inverse of the examples they have.

*setFeatureScaling(string)* - Normalizes the feature factors using the specified method ("zscore", "minmax" or empty for no normalization).

*setOutputLogsPath(string)* - Sets the path to a folder where logs of training progress will be saved. No logs are generated if no path is specified.

In [0]:
spark_df = spark.createDataFrame(df)
spark_df.select(spark_df.columns[-10:]).show(2)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Sparse pandas data (column col_0) not supported.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
col_293|col_294|col_295|col_296|col_297|col_298| col_299| col_300| col_301|target|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|-0.4524794726808656| 0.9502875792756131|-0.8297616989552165| 1|
 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| -0.555981017719065|-0.29938816657135553|-0.5118709929431844| 1|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
only showing top 2 rows

In [0]:
(training_data, test_data) = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 9185
Test Dataset Count: 2352

## Create a custom DL architecture with TF

In [0]:
# for a custom graph

'''
from sparknlp_jsl.training import tf_graph

#!mkdir gc_graph

tf_graph.print_model_params("generic_classifier")
'''

Out[66]: '\nfrom sparknlp_jsl.training import tf_graph\n\n#!mkdir gc_graph\n\ntf_graph.print_model_params("generic_classifier")\n'

In [0]:
# for a custom graph
'''
DL_params = {"input_dim": input_dim, 
             "output_dim": y.nunique(), 
             "hidden_layers": [300, 200, 100], 
             "hidden_act": "tanh",
             'hidden_act_l2':1,
             'batch_norm':1}


tf_graph.build("generic_classifier",build_params=DL_params, model_location="file:/databricks/driver/gc_graph", model_filename="auto")
'''

In [0]:
#or just use the one we already have in the repo

! mkdir /databricks/driver/generic_logs
! mkdir /databricks/driver/gc_graph

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /databricks/driver/gc_graph

In [0]:
%fs ls file:/databricks/driver/gc_graph

path,name,size
file:/databricks/driver/gc_graph/pet.in1202D.out2.pb,pet.in1202D.out2.pb,122582


In [0]:
from sparknlp_jsl.base import *

#!mkdir logs

features_asm = FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile('/databricks/driver/gc_graph/pet.in1202D.out2.pb')\
    .setEpochsNumber(100)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("file:/databricks/driver/generic_logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])



In [0]:
# train 100 epochs (takes around 37 seconds)

clf_model = clf_Pipeline.fit(training_data)


In [0]:
%sh cd generic_logs && ls -lt

total 8
-rw-r--r-- 1 root root 7757 Jul 10 15:59 GenericClassifierApproach_45c44871d5cb.log

In [0]:
%sh

cat generic_logs/GenericClassifierApproach_45c44871d5cb.log

Training 100 epochs
Epoch 1/100	0.36s	Loss: 18.615316	ACC: 0.63432443	Validation ACC: 0.72182906
Epoch 2/100	0.21s	Loss: 15.819245	ACC: 0.7161824	Validation ACC: 0.71257484
Epoch 3/100	0.21s	Loss: 13.479971	ACC: 0.7778266	Validation ACC: 0.6880784
Epoch 4/100	0.22s	Loss: 10.914008	ACC: 0.82082206	Validation ACC: 0.7196516
Epoch 5/100	0.21s	Loss: 8.175004	ACC: 0.8734461	Validation ACC: 0.68209034
Epoch 6/100	0.22s	Loss: 5.832642	ACC: 0.92045027	Validation ACC: 0.66303754
Epoch 7/100	0.22s	Loss: 4.335216	ACC: 0.9425223	Validation ACC: 0.7071312
Epoch 8/100	0.22s	Loss: 2.9919016	ACC: 0.9652365	Validation ACC: 0.6989657
Epoch 9/100	0.22s	Loss: 2.3633766	ACC: 0.970676	Validation ACC: 0.7310833
Epoch 10/100	0.22s	Loss: 2.141851	ACC: 0.97593474	Validation ACC: 0.71801853
Epoch 11/100	0.22s	Loss: 1.7258862	ACC: 0.97810805	Validation ACC: 0.72019595
Epoch 12/100	0.21s	Loss: 1.4819793	ACC: 0.98201597	Validation ACC: 0.7191072
Epoch 13/100	0.21s	Loss: 1.4026744	ACC: 0.98405445	Validation ACC: 0.73326075
Epoch 14/100	0.21s	Loss: 1.1274608	ACC: 0.9870049	Validation ACC: 0.7005988
Epoch 15/100	0.21s	Loss: 0.9198623	ACC: 0.9894597	Validation ACC: 0.7283615
Epoch 16/100	0.21s	Loss: 0.5228828	ACC: 0.9932211	Validation ACC: 0.7338051
Epoch 17/100	0.21s	Loss: 1.0413896	ACC: 0.9872976	Validation ACC: 0.73162764
Epoch 18/100	0.22s	Loss: 0.88151133	ACC: 0.9897187	Validation ACC: 0.7310833
Epoch 19/100	0.21s	Loss: 0.9871707	ACC: 0.98685825	Validation ACC: 0.73326075
Epoch 20/100	0.21s	Loss: 1.2445159	ACC: 0.9867569	Validation ACC: 0.726184
Epoch 21/100	0.22s	Loss: 0.8328553	ACC: 0.9901354	Validation ACC: 0.7512248
Epoch 22/100	0.22s	Loss: 1.2573304	ACC: 0.9855294	Validation ACC: 0.74904734
Epoch 23/100	0.22s	Loss: 1.1562771	ACC: 0.9882434	Validation ACC: 0.7245509
Epoch 24/100	0.22s	Loss: 0.79088795	ACC: 0.99108136	Validation ACC: 0.72999454
Epoch 25/100	0.22s	Loss: 1.5994883	ACC: 0.98593503	Validation ACC: 0.7397931
Epoch 26/100	0.21s	Loss: 0.9635311	ACC: 0.99000025	Validation ACC: 0.7495917
Epoch 27/100	0.21s	Loss: 0.73284096	ACC: 0.9927028	Validation ACC: 0.75557977
Epoch 28/100	0.21s	Loss: 0.75912994	ACC: 0.99202716	Validation ACC: 0.7387044
Epoch 29/100	0.21s	Loss: 0.7297018	ACC: 0.99364895	Validation ACC: 0.74904734
Epoch 30/100	0.21s	Loss: 0.30958676	ACC: 0.99567586	Validation ACC: 0.74305934
Epoch 31/100	0.21s	Loss: 0.48060983	ACC: 0.9944596	Validation ACC: 0.7414262
Epoch 32/100	0.22s	Loss: 0.41734135	ACC: 0.9941892	Validation ACC: 0.74305934
Epoch 33/100	0.21s	Loss: 0.69376135	ACC: 0.9927029	Validation ACC: 0.7359826
Epoch 34/100	0.22s	Loss: 0.6752465	ACC: 0.9894596	Validation ACC: 0.7495917
Epoch 35/100	0.21s	Loss: 0.9120156	ACC: 0.9897187	Validation ACC: 0.7196516
Epoch 36/100	0.21s	Loss: 0.8426922	ACC: 0.99025923	Validation ACC: 0.7419706
Epoch 37/100	0.22s	Loss: 0.4953444	ACC: 0.9936375	Validation ACC: 0.73326075
Epoch 38/100	0.21s	Loss: 0.5930925	ACC: 0.99309707	Validation ACC: 0.726184
Epoch 39/100	0.21s	Loss: 0.44531333	ACC: 0.99418926	Validation ACC: 0.726184
Epoch 40/100	0.22s	Loss: 0.482242	ACC: 0.99390763	Validation ACC: 0.7474143
Epoch 41/100	0.22s	Loss: 1.1897033	ACC: 0.98552954	Validation ACC: 0.7283615
Epoch 42/100	0.21s	Loss: 0.6457009	ACC: 0.9925677	Validation ACC: 0.73326075
Epoch 43/100	0.21s	Loss: 0.60482997	ACC: 0.99282694	Validation ACC: 0.7343495
Epoch 44/100	0.21s	Loss: 0.39960185	ACC: 0.994865	Validation ACC: 0.72182906
Epoch 45/100	0.21s	Loss: 0.37690184	ACC: 0.9947298	Validation ACC: 0.7376157
Epoch 46/100	0.21s	Loss: 0.43632263	ACC: 0.99539423	Validation ACC: 0.72999454
Epoch 47/100	0.21s	Loss: 0.25911814	ACC: 0.996892	Validation ACC: 0.72945017
Epoch 48/100	0.22s	Loss: 0.48160508	ACC: 0.99500006	Validation ACC: 0.73816
Epoch 49/100	0.22s	Loss: 0.4816355	ACC: 0.9944596	Validation ACC: 0.7354382
Epoch 50/100	0.21s	Loss: 0.39345434	ACC: 0.99472994	Validation ACC: 0.73326075
Epoch 51/100	0.22s	Loss: 0.8282523	ACC: 0.99108136	Validation ACC: 0.7147523
Epoch 52/100	0.21s	Loss: 0.77980393	ACC: 0.99297315	Validation AC

In [0]:
pred_df = clf_model.transform(test_data)

In [0]:
pred_df.select('target','prediction.result').show()

+------+------+
target|result|
+------+------+
 1| [1]|
 1| [0]|
 1| [1]|
 1| [1]|
 1| [1]|
 0| [0]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [1]|
 1| [0]|
 0| [1]|
 1| [0]|
+------+------+
only showing top 20 rows

In [0]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))


In [0]:
preds_df

Out[46]:

,target,result
0,1,1
1,1,0
2,1,1
3,1,1
4,1,1
...,...,...
2347,0,1
2348,1,0
2349,1,0
2350,1,1


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['target'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['target']))


precision recall f1-score support

 0 0.4615 0.5036 0.4816 560
 1 0.8403 0.8164 0.8282 1792

 accuracy 0.7419 2352
 macro avg 0.6509 0.6600 0.6549 2352
weighted avg 0.7501 0.7419 0.7457 2352

0.7419217687074829

# Case Study: Alexa Review Classification

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/amazon_alexa.tsv -P /dbfs/

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/amazon_alexa.tsv', sep='\t')
df

Out[53]:

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wr...",1
3,5,31-Jul-18,Charcoal Fabric,"I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs, i control the light...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in between!!",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, checking time, looking up weather. There are many more ...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my entire home, TV, all my lights, my thermostat, my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound quality isn't great. I mostly use it for commands though...,1


In [0]:
df.verified_reviews = df.verified_reviews.str.lower()

In [0]:
df.feedback.value_counts()

Out[55]: 1 2893
0 257
Name: feedback, dtype: int64

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (TfidfVectorizer(max_features=1000,  norm='l2', ngram_range=(1, 3)), 'verified_reviews'))

X = column_trans.fit_transform(df.drop(['feedback'], axis=1))

y = df.feedback

In [0]:
import scipy.sparse

sdf = pd.DataFrame.sparse.from_spmatrix(X)

sdf.columns = ['col_{}'.format(i) for i in range(X.shape[1])]

sdf = sdf.fillna(0)

sdf['feedback']= y

sdf.head()

Out[57]:

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_951,col_952,col_953,col_954,col_955,col_956,col_957,col_958,col_959,col_960,col_961,col_962,col_963,col_964,col_965,col_966,col_967,col_968,col_969,col_970,col_971,col_972,col_973,col_974,col_975,col_976,col_977,col_978,col_979,col_980,col_981,col_982,col_983,col_984,col_985,col_986,col_987,col_988,col_989,col_990,col_991,col_992,col_993,col_994,col_995,col_996,col_997,col_998,col_999,feedback
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145963,0.146283,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.327234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.324472,0.0,0.149662,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.16371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.104466,0.0,0.0,0.0,0.0,0.0,0.210616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1


In [0]:
input_spark_df = spark.createDataFrame(sdf)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:300: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Sparse pandas data (column col_0) not supported.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
(training_data, test_data) = input_spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 2478
Test Dataset Count: 672

In [0]:
%sh cd /databricks/driver/gc_graph && ls -la

total 128
drwxr-xr-x 2 root root 4096 Jul 10 15:58 .
drwxr-xr-x 1 root root 4096 Jul 10 15:58 ..
-rw-r--r-- 1 root root 122582 Jul 10 15:58 pet.in1202D.out2.pb

In [0]:
from sparknlp_jsl.base import *

features_asm = FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = GenericClassifierApproach()\
    .setLabelColumn("feedback")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("file:/databricks/driver/generic_logs")\
   #.setModelFile("/databricks/driver/gc_graph/pet_in1202D_out2.pb")
    

clf_Pipeline = Pipeline(stages=[
    features_asm, 
    gen_clf])

clf_model = clf_Pipeline.fit(training_data)


In [0]:
pred_df = clf_model.transform(test_data)

preds_df = pred_df.select('feedback','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['feedback'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['feedback']))


precision recall f1-score support

 0 0.5690 0.5593 0.5641 59
 1 0.9577 0.9592 0.9584 613

 accuracy 0.9241 672
 macro avg 0.7633 0.7593 0.7613 672
weighted avg 0.9235 0.9241 0.9238 672

0.9241071428571429